In [1]:
import seaborn as sns

mpg = sns.load_dataset('mpg')

In [2]:
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [3]:
live_data = [
    [350, 165, 3693],
    [120, 120, 1200],
    [318, 150, 3000]
]

In [4]:
features = mpg[['displacement', 'horsepower', 'weight']]
target = mpg['mpg']

In [5]:
from sklearn.linear_model import SGDRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [6]:
# Custom Transformers -> Column -> module
# import custom function and transformers
model = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scaling', StandardScaler()),
    ('sgd model', SGDRegressor())
])

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(features, target)

In [9]:
model.fit(X_train, y_train)

Pipeline(steps=[('impute', SimpleImputer(strategy='median')),
                ('scaling', StandardScaler()), ('sgd model', SGDRegressor())])

In [10]:
pred = model.predict(X_test)

In [11]:
from sklearn.metrics import r2_score

In [12]:
print(r2_score(y_test, pred))

0.6408727813888143


In [13]:
import joblib

In [14]:
joblib.dump(model, 'mpg_sgd.pkl')

['mpg_sgd.pkl']

In [15]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
import joblib
import numpy as np

In [17]:
model = joblib.load("mpg_sgd.pkl")

In [18]:
import numpy as np

In [19]:
live_data = np.array([
    [350, 165, 3693],
    [120, 120, np.nan],
    [318, 150, 3000]
])

In [20]:
preds = model.predict(live_data)

C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


In [21]:
model.named_steps['impute'].transform([[np.nan, np.nan, np.nan]])

C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([[ 151. ,   92.5, 2834. ]])

In [22]:
print(preds)

[16.71018428 23.45288054 21.41464955]


In [23]:
%%writefile mpg_predictor.py
import tkinter as tk
import joblib
import numpy as np

model = joblib.load("mpg_sgd.pkl") 


root = tk.Tk()

frame1 = tk.Frame(root)
label1 = tk.Label(frame1, text="Displacement".center(30)+" : ",
        width=20
        )
label1.config(font=('monospace', 30, 'bold'), fg='#333333', bg='#eeeeee')
label1.pack(expand=tk.YES,padx=15, pady=15,side=tk.LEFT)

dis = tk.StringVar()
dis.set('')
entry1 = tk.Entry(frame1, textvariable=dis,
        width=20
        )

entry1.focus()
entry1.config(font=('monospace', 30, 'bold'), fg='#333333', bg='#eeeeee')
entry1.pack(expand=tk.YES,padx=15, pady=15,side=tk.LEFT)
frame1.pack(expand=tk.YES,padx=15, pady=15,)

frame2 = tk.Frame(root)
label2 = tk.Label(frame2, text="Horse Power".center(30)+" : ", width=20)
label2.config(font=('monospace', 30, 'bold'), fg='#333333', bg='#eeeeee')
label2.pack(expand=tk.YES,padx=15, pady=15,side=tk.LEFT,fill=tk.X)

horse = tk.StringVar()
horse.set('')

entry2 = tk.Entry(frame2, textvariable=horse, width=20)
entry2.config(font=('monospace', 30, 'bold'), fg='#333333', bg='#eeeeee')
entry2.pack(expand=tk.YES,padx=15, pady=15,side=tk.LEFT)
frame2.pack(expand=tk.YES,padx=15, pady=15,)

frame3 = tk.Frame(root)
label3 = tk.Label(frame3, text="Weight".center(30)+"      : ", width=20)
label3.config(font=('monospace', 30, 'bold'), fg='#333333', bg='#eeeeee')
label3.pack(expand=tk.YES,padx=15, pady=15,side=tk.LEFT)

weight = tk.StringVar()
weight.set('')

entry3 = tk.Entry(frame3, textvariable=weight, width=20)
entry3.config(font=('monospace', 30, 'bold'), fg='#333333', bg='#eeeeee')
entry3.pack(expand=tk.YES,padx=15, pady=15,side=tk.LEFT, fill=tk.X)
frame3.pack(expand=tk.YES,padx=15, pady=15,)



def predict():
    d = dis.get()
    if d:
        d = float(d)
    else:
        d = np.nan
    h = horse.get()
    if h:
        h = float(h)
    else:
        h = np.nan
    w = weight.get()
    if w:
        w = float(w)
    else:
        w = np.nan
    pred = round(model.predict([[d, h, w], ])[0])
    dis.set('')
    horse.set('')
    weight.set('')
    win = tk.Toplevel(root)
    win.grab_set()
    ans = f"""
    ___________________
    HorsePower    = {h:.2f}
    Weight        = {w:.2f}
    Displacement  = {d:.2f}
    ___________________
    Milege        = {pred:.2f}
    """
    lb = tk.Message(win, text=ans)
    lb.config(font=("monospace", 20, 'bold'),
            fg='#eeeeee',
            bg='#A6AAFF',
            )
    lb.pack(expand=tk.YES,padx=15, pady=15,fill=tk.BOTH)
    b = tk.Button(win, text='EXIT', command=lambda :(win.destroy(), entry1.focus()))
    b.config(font=("monospace", 20, 'bold'),
            fg='red',
            bg='#eeeeee',
    )
    b.pack(expand=tk.YES,padx=15, pady=15,fill=tk.X)

button = tk.Button(root, text='Predict', command=lambda : predict())
button.config(font=('Times', 30, 'bold'), fg='red', bg='white')
button.pack(expand=tk.YES,padx=15, pady=15, fill=tk.X)

button1 = tk.Button(root, text='EXIT', command=lambda : root.quit())
button1.config(font=('Times', 30, 'bold'), bg='red', fg='white')
button1.pack(expand=tk.YES,padx=15, pady=15,side=tk.BOTTOM, fill=tk.X)
root.title('Milege Prediction')
root.mainloop()

Writing mpg_predictor.py


In [24]:
!python mpg_predictor.py

C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
